In [126]:
import os
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from zipfile import ZipFile

# Load the data

In [151]:
DATA_PATH = "../00_source_data"
arrests_data = os.path.join(DATA_PATH, 'ucr_arrests_monthly_all'
                                       '_crimes_race_sex_1974_2020_dta.zip')

zip_file = ZipFile(arrests_data)
dfs = {text_file.filename: pd.read_stata(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.dta')}
zip_file.close()

In [152]:
print(dfs.keys())
arrests_2019 = dfs['ucr_arrests_monthly_all_crimes_race_sex_2019.dta'].copy()
arrests_2020 = dfs['ucr_arrests_monthly_all_crimes_race_sex_2020.dta'].copy()
print(arrests_2019.shape)
print(arrests_2020.shape)

dict_keys(['ucr_arrests_monthly_all_crimes_race_sex_2019.dta', 'ucr_arrests_monthly_all_crimes_race_sex_2020.dta'])
(233476, 1227)
(236468, 1233)


In [153]:
zip_file = ZipFile(os.path.join(DATA_PATH, 'crime.zip'))
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}
zip_file.close()

In [154]:
print(dfs.keys())
crime_data = dfs['crime.csv'].copy()
offense_codes = dfs['offense_codes.csv'].copy()
print(crime_data.shape)
print(offense_codes.shape)

dict_keys(['crime.csv', 'offense_codes.csv'])
(473344, 19)
(299, 9)


# Preprocessing

In [155]:
arrests_2019_cols = set(arrests_2019.columns)
arrests_2020_cols = set(arrests_2020.columns)
print(arrests_2019_cols.difference(arrests_2020_cols))
print("*" * 50)
print(arrests_2020_cols.difference(arrests_2019_cols))
print("*" * 50)
print(len(arrests_2020_cols.intersection(arrests_2019_cols)))

set()
**************************************************
{'address_city', 'address_state', 'address_street_line_1', 'address_street_line_2', 'address_name', 'address_zip_code'}
**************************************************
1227


In [156]:
columns_to_remove = list(arrests_2020_cols.difference(arrests_2019_cols))
arrests_2020.drop(columns_to_remove, axis=1, inplace=True)
assert arrests_2019.shape[1] == arrests_2020.shape[1]

In [157]:
arrests_2020.head()

,ori,ori9,population,agency_name,year,month,state,state_abb,population_group,country_division,...,weapons_tot_male,weapons_tot_male_adult,weapons_tot_male_juv,weapons_tot_non_hisp,weapons_tot_white,juv_handled_within_department,juv_referred_to_juv_court,juv_referred_to_welfare,juv_referred_to_police,juv_referred_to_crim_court
0,AK00101,AK0010100,286388,anchorage,2020,april,alaska,AK,"city 250,000 thru 499,999",pacific,...,13,12,1,0,9,0,0,0,0,0
1,AK00101,AK0010100,286388,anchorage,2020,august,alaska,AK,"city 250,000 thru 499,999",pacific,...,6,4,2,0,2,0,0,0,0,0
2,AK00101,AK0010100,286388,anchorage,2020,december,alaska,AK,"city 250,000 thru 499,999",pacific,...,6,6,0,0,3,0,0,0,0,0
3,AK00101,AK0010100,286388,anchorage,2020,february,alaska,AK,"city 250,000 thru 499,999",pacific,...,11,8,3,0,6,0,0,0,0,0
4,AK00101,AK0010100,286388,anchorage,2020,january,alaska,AK,"city 250,000 thru 499,999",pacific,...,14,13,1,0,7,0,0,0,0,0


In [160]:
arrests_2020_filtered = arrests_2020.query("state_abb == 'CO'").copy()
arrests_2019_filtered = arrests_2019.query("state_abb == 'CO'").copy()

In [161]:
arrests_2020.loc[(arrests_2020['fips_state_county_code'] == '') &
                 (arrests_2020['agency_name'] == 'us secret service, denve'), 'fips_state_county_code'] = '08031'

In [162]:
arrests_2019.loc[(arrests_2019['fips_state_county_code'] == '') &
                 (arrests_2019['agency_name'] == 'us secret service, denve'), 'fips_state_county_code'] = '08031'

In [163]:
arrests_2020_filtered = arrests_2020_filtered.query("fips_state_county_code in ['08031', '08013', '08014']").copy()
arrests_2019_filtered = arrests_2019_filtered.query("fips_state_county_code in ['08031', '08013', '08014']").copy()
print(arrests_2020_filtered.shape)
print(arrests_2019_filtered.shape)

(144, 1227)
(156, 1227)


In [167]:
general_cols = arrests_2020_filtered.columns.to_list()[:14]
arrest_cols = [col for col in arrests_2020_filtered.columns if 'tot_arrest' in col]
arrests_2020_filtered = arrests_2020_filtered[general_cols + arrest_cols]
arrests_2019_filtered = arrests_2019_filtered[general_cols + arrest_cols]

In [168]:
arrests_df = pd.concat([arrests_2019_filtered, arrests_2020_filtered], axis=0)
print(arrests_df.shape)

(300, 57)


In [169]:
arrests_df.head()

,ori,ori9,population,agency_name,year,month,state,state_abb,population_group,country_division,...,sale_heroin_coke_tot_arrests,sale_other_drug_tot_arrests,sale_synth_narc_tot_arrests,stolen_prop_tot_arrests,suspicion_tot_arrests,theft_tot_arrests,total_drug_tot_arrests,vagrancy_tot_arrests,vandalism_tot_arrests,weapons_tot_arrests
14839,CO00700,CO0070000,58601,boulder,2019,april,colorado,CO,"msa-county 25,000 thru 99,999",mountain,...,2,0,0,0,0,9,18,0,2,0
14840,CO00700,CO0070000,58601,boulder,2019,august,colorado,CO,"msa-county 25,000 thru 99,999",mountain,...,2,0,0,0,0,6,8,0,3,0
14841,CO00700,CO0070000,58601,boulder,2019,december,colorado,CO,"msa-county 25,000 thru 99,999",mountain,...,3,0,0,0,0,12,8,0,5,0
14842,CO00700,CO0070000,58601,boulder,2019,february,colorado,CO,"msa-county 25,000 thru 99,999",mountain,...,2,0,0,0,0,2,9,0,5,0
14843,CO00700,CO0070000,58601,boulder,2019,january,colorado,CO,"msa-county 25,000 thru 99,999",mountain,...,0,0,0,0,0,2,9,0,0,0


In [170]:
arrests_agg = (arrests_df
               .groupby(['month', 'year', 'fips_state_county_code'], as_index=False)[arrest_cols].sum())

In [178]:
population_norm = (arrests_df
                   .groupby(['month', 'year', 'fips_state_county_code', 'fips_place_code'], as_index=False)[
                       'population']
                   .max()
                   .groupby(['month', 'year', 'fips_state_county_code'], as_index=False)[['population']]
                   .sum())

In [180]:
merged = pd.merge(arrests_agg, population_norm, on=['month', 'year', 'fips_state_county_code'])
print(merged.shape)

(72, 47)


In [181]:
merged.head()

,month,year,fips_state_county_code,agg_assault_tot_arrests,all_other_tot_arrests,arson_tot_arrests,burglary_tot_arrests,curfew_loiter_tot_arrests,disorder_cond_tot_arrests,drunkenness_tot_arrests,...,sale_other_drug_tot_arrests,sale_synth_narc_tot_arrests,stolen_prop_tot_arrests,suspicion_tot_arrests,theft_tot_arrests,total_drug_tot_arrests,vagrancy_tot_arrests,vandalism_tot_arrests,weapons_tot_arrests,population
0,april,2019,08013,18,532,1,12,0,24,0,...,0,0,3,0,105,57,0,22,5,317073
1,april,2019,08014,1,31,0,2,0,1,0,...,4,0,0,0,43,26,0,5,0,70798
2,april,2019,08031,143,848,1,38,26,62,0,...,5,1,12,0,196,350,2,98,73,728941
3,april,2020,08013,23,179,7,21,0,9,0,...,1,0,0,0,71,9,0,19,3,315516
4,april,2020,08014,1,11,0,0,0,0,0,...,0,0,1,0,18,3,0,3,1,71795


In [182]:
violent_crime = ['agg_assault_tot_arrests', 'arson_tot_arrests', 'burglary_tot_arrests',
                 'manslaught_neg_tot_arrests', 'murder_tot_arrests', 'oth_assault_tot_arrests',
                 'oth_sex_off_tot_arrests', 'rape_tot_arrests', 'robbery_tot_arrests']
non_violent = list(set(arrest_cols).difference(set(violent_crime)))
print(len(violent_crime))
print(len(non_violent))
print(len(arrest_cols))

9
34
43


# EDA

In [185]:
merged.head()

,month,year,fips_state_county_code,agg_assault_tot_arrests,all_other_tot_arrests,arson_tot_arrests,burglary_tot_arrests,curfew_loiter_tot_arrests,disorder_cond_tot_arrests,drunkenness_tot_arrests,...,sale_other_drug_tot_arrests,sale_synth_narc_tot_arrests,stolen_prop_tot_arrests,suspicion_tot_arrests,theft_tot_arrests,total_drug_tot_arrests,vagrancy_tot_arrests,vandalism_tot_arrests,weapons_tot_arrests,population
0,april,2019,08013,18,532,1,12,0,24,0,...,0,0,3,0,105,57,0,22,5,317073
1,april,2019,08014,1,31,0,2,0,1,0,...,4,0,0,0,43,26,0,5,0,70798
2,april,2019,08031,143,848,1,38,26,62,0,...,5,1,12,0,196,350,2,98,73,728941
3,april,2020,08013,23,179,7,21,0,9,0,...,1,0,0,0,71,9,0,19,3,315516
4,april,2020,08014,1,11,0,0,0,0,0,...,0,0,1,0,18,3,0,3,1,71795
